In [ ]:
import yfinance as yf
import pandas as pd

# Tickers and company names for fast-food restaurants
fast_food_ticker_company_dict = {
    'MCD': 'McDonald\'s',
    'DPZ': 'Domino\'s Pizza',
    'YUM': 'Yum! Brands',
    'WEN': 'Wendy\'s',
    'CMG': 'Chipotle',
    'SHAK': 'Shake Shack',
    'JACK': 'Jack in the Box',
    'PZZA': 'Papa John\'s'
}

# Tickers and company names for other restaurants
other_restaurant_ticker_company_dict = {
    'CAKE': 'The Cheesecake Factory',
    'TXRH': 'Texas Roadhouse',
    'BJRI': 'BJ\'s Restaurant',
    'RRGB': 'Red Robin',
    'DENN': 'Denny\'s',
    'EAT': 'Chilis',
    'DRI': 'Olive Garden',
    'CBRL': 'Cracker Barrel'
}

def format_as_dollars(value):
    if value is None:
        return 'N/A'
    return '${:,.2f}'.format(value)

def get_annual_sales(ticker_company_dict):
    sales_data = []

    for ticker, company in ticker_company_dict.items():
        stock = yf.Ticker(ticker)
        income_statement = stock.financials
        if 'Total Revenue' in income_statement.index:
            annual_sales = income_statement.loc['Total Revenue']
            sales_data.append({'Company': company, 'Ticker': ticker, 'Annual Sales': format_as_dollars(annual_sales.iloc[0])})
        else:
            sales_data.append({'Company': company, 'Ticker': ticker, 'Annual Sales': 'N/A'})

    return pd.DataFrame(sales_data)

# Create DataFrames for both sets of companies
fast_food_sales_df = get_annual_sales(fast_food_ticker_company_dict)
other_restaurant_sales_df = get_annual_sales(other_restaurant_ticker_company_dict)

# Display the DataFrames with dollar formatting for 'Annual Sales' column
pd.options.display.float_format = '${:,.2f}'.format

print("Fast-Food Restaurants Annual Sales:")
print(fast_food_sales_df)
print("\nOther Restaurants Annual Sales:")
print(other_restaurant_sales_df)

# Calculate the sum of annual sales for each DataFrame
fast_food_total_sales = fast_food_sales_df['Annual Sales'].replace('[\$,]', '', regex=True).astype(float).sum()
other_restaurant_total_sales = other_restaurant_sales_df['Annual Sales'].replace('[\$,]', '', regex=True).astype(float).sum()

print("\nTotal Annual Sales for Fast-Food Restaurants: ", format_as_dollars(fast_food_total_sales))
print("Total Annual Sales for Other Restaurants: ", format_as_dollars(other_restaurant_total_sales))

In [ ]:
#Last Ran 8/2/2023

import requests
import pandas as pd
import time
import os
from config import api_key

# Replace 'YOUR_API_KEY' with your actual Yelp API key
API_KEY = 'E9tLn7C9Nmzq2m8OeZBERlrJH1EUE1s0wvM559mntHgRtzjwEfVCcOTVJL6mphnvvSwzk303N342ddthgrWhBwOArWfvg6VBM2hu1D6pYt-r8E0VeCC1nOyLNHXIZHYx'


def get_restaurant_ratings(restaurant_list, state_list):
    endpoint = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
    }
    restaurant_data = {"Restaurant": [], "Mean Rating": []}

    for restaurant in restaurant_list:
        restaurant_ratings = []

        for state in state_list:
            params = {
                "term": restaurant,
                "location": state,
                "limit": 50,  # Adjust the limit based on the number of results you want to retrieve for each state
            }

            response = requests.get(endpoint, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()

                if "businesses" in data:
                    for business in data["businesses"]:
                        rating = business["rating"]
                        restaurant_ratings.append(rating)

                else:
                    print(f"No data for {restaurant} in {state}")

            elif response.status_code == 429:
                print(f"API rate limit reached. Waiting for 60 seconds...")
                time.sleep(60)
                continue

            else:
                print(f"Error fetching data for {restaurant} in {state}. Status Code: {response.status_code}")

            # Introduce a delay of 2 seconds to avoid API rate limit issues
            time.sleep(2)

        mean_rating = calculate_mean(restaurant_ratings)
        if mean_rating is not None:
            restaurant_data["Restaurant"].append(restaurant)
            restaurant_data["Mean Rating"].append(mean_rating)

    return restaurant_data

def calculate_mean(ratings):
    if not ratings:
        return None
    return sum(ratings) / len(ratings)

if __name__ == "__main__":
    fast_food = [
        "McDonald's", "In-N-Out", "Taco Bell", "Wendy's", "Chipotle",
        "Shake Shack", "Popeyes", "Jack in the Box", "Chick-fil-A", "Papa John's"#,
        #"Domino's", "Panda Express", "Pizza Hut", "Burger King", "KFC", "Dairy Queen", "Subway"
    ]

    restaurants_to_check = [
        "Cheesecake Factory", "Texas Roadhouse", "BJ's Restaurant", "Red Robin", "Denny's",
        "Golden Corral", "Applebees", "Chilis", "Olive Garden", "Cracker Barrel"#,
        #"Outback Steakhouse", "Hard Rock Cafe", "Longhorn Steakhouse",
        #"IHOP", "Yard House", "P.F. Chang's", "Red Lobster"
    ]

    # List of all 50 states in the United States
    state_list = [
        "California", "Texas", "Florida", "New York", "Pennsylvania",
        "Illinois", "Ohio", "Georgia", "North Carolina", "Michigan"
    ]

    # Process fast-food restaurants
    fast_food_data = get_restaurant_ratings(fast_food, state_list)
    fast_food_df = pd.DataFrame(fast_food_data)
    print(fast_food_df)

    # Process other restaurants
    other_restaurants_data = get_restaurant_ratings(restaurants_to_check, state_list)
    other_restaurants_df = pd.DataFrame(other_restaurants_data)
    print(other_restaurants_df)
    '''
    # Merge the dataframes side by side
    combined_df = pd.merge(fast_food_df, other_restaurants_df, on='Restaurant', suffixes=('_fast_food', '_other'))

    # Save the combined dataframe to CSV
    output_folder = "output_data"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    combined_df.to_csv(os.path.join(output_folder, "clean_yelp.csv"), index=False)

    print("CSV file created: clean_yelp.csv")
    '''

    ### fix issue with not printing anything to csv or anything. for some reason im getting partials
    ### but not the whole thing. silly